For Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data preprocessing with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    '/home/raone/IP_idcard_classifier/train_directory',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    '/home/raone/IP_idcard_classifier/train_directory',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=250//32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=250//32,
    callbacks=[early_stop]
)

# Save the model
model.save('NEW.h5')
print("Model saved successfully!")


For Running the Classification Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load the saved model
model = tf.keras.models.load_model('NEW.h5')

# Define a function to predict the class of an image
def predict_image_class(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    if predictions[0] > 0.5:
        return 'Valid ID Card'
    else:
        return 'Invalid ID Card'

# Test the model on a sample image
image_path = '/home/raone/IP_idcard_classifier/Screenshot from 2024-04-14 16-10-02.png'  # Replace with the path to your test image
predicted_class = predict_image_class(image_path)

# Display the test image and predicted class
img = image.load_img(image_path, target_size=(150, 150))
plt.imshow(img)
plt.title(predicted_class)
plt.axis('off')
plt.show()

print(f"The predicted class of the image is: {predicted_class}")
